# PCR Primer design
So far we have used Snapgene to design our primers. However in order to explore open-source alternatives this notebook will use `Biopython` in conjunction with Primer3
## Installation
First we need to install the python wrapper `primer3-py`

In [1]:
!pip install primer3-py

   ---------------------------------------- 0.0/1.6 MB ? eta -:--:--
   -------------------------- ------------- 1.0/1.6 MB ? eta -:--:--
   -------------------------- ------------- 1.0/1.6 MB ? eta -:--:--
   -------------------------- ------------- 1.0/1.6 MB ? eta -:--:--
   -------------------------- ------------- 1.0/1.6 MB ? eta -:--:--
   ---------------------------------------- 1.6/1.6 MB 1.3 MB/s  0:00:01


## Import
We can now import it along with other libraries we need

In [4]:
from Bio import SeqIO
import primer3
import os

## Designing our first primer

In [ ]:
dna_seq = str("AGCGCACATCTCTAATCTTTGATTTACGTATAATCGTTACACAACCATCCCCCTAATTCCTCTACAGAGGCGCGTCCTGCTGCCCCTTCCCATGGCACATGCTTCCAACTTACGATTTCTCACGTCCCACTCGTCCCAGGTTTCGCCTTTTCGGATTTCGTCCGTGTTGTGTTTTGTCTTCTTCCGTTGTCTTCCTCCGTTGTCTTCGATTTGATTCCCCACACCCGACTGCCTATCATTGTTCTTGCTTCGGATTCCTGGGCCACCTCGTTGAACTGGTGCTACTCAATAATTTATCGTAATTTACCATACGTTATCGATACATACTCCCCCCCATTCCGTTGTATTCCTTCGCTGTAGCTTTTGAGACCACCACCACCACCCTAGCCTCTGTTGTATAGTTTTCGACTTCGTCCGCCCGCCGTTGTCCTTGTGAACCATTTTTGTTGCTATCGCACACATACATACCAAGTACACTGCTTCTTGCTATCCTCCCTACTAGCTTTCTACATGCATTGCTACTATACAGCCCTCTACAGCACTTGAAATGAAGTAATTTGAGCCCGGAGCATCCACGAAACCTACGCGAGGAAGAGGTTCTTTAGGCGCTGGTCTGCATTTTCGCCCATACTCAGTCCCAAATTCCAGGGAACGAGGCCAGTAATACTACTGTATTGATTGTCGTTTTACAGCCACTGTCACTGTCATTCAATGTCAACGAACGGCGTCCAGACTTGTCAAGCATCGTGCCAACGATTTGTTTCCGCGTTCACAGCATCCGCTCCAACATCTCCCTATTTTCGACGTACAAATCAAACCCTAGTTGACAGCAAACCTAACGGTAGTGTTGTTGCGAGGTGGAGGTCCCTCAAGGTGGAGTTACCTTGAGGTCCCGTATTTGTGATGGTGGCGTTCTCAAGGTCCCTTGAGGTGGCCTTGATGTGTTTTCCCATCCCTGTAACGGAATCTCCGGGACGGTGATGTGGCAGCGAGTGCGAGC")

primers = primer3.bindings.design_primers({'SEQUENCE_TEMPLATE': dna_seq,}, {'PRIMER_OPT_SIZE': 20,'PRIMER_MIN_SIZE': 18, 'PRIMER_MAX_SIZE': 25, 'PRIMER_OPT_TM': 60.0, 'PRIMER_MIN_TM': 57.0, 'PRIMER_MAX_TM': 63.0, 'PRIMER_PAIR_MAX_DIFF_TM': 3.0, 'PRIMER_PRODUCT_SIZE_RANGE': [[900, 1100]],'SEQUENCE_INCLUDED_REGION': [0, len(dna_seq)],'SEQUENCE_PRIMER_PAIR_OK_REGION_LIST': [[0, 25, len(dna_seq)-25, 25]]})

# Print the first primer pair
print("Forward primer:", primers['PRIMER_LEFT_0_SEQUENCE'])
print("Reverse primer:", primers['PRIMER_RIGHT_0_SEQUENCE'])
print("Product size:", primers['PRIMER_PAIR_0_PRODUCT_SIZE'])

Forward primer: AGCGCACATCTCTAATCTTTGA
Reverse primer: ACTCGCTGCCACATCACC
Product size: 994


As we can see the primer is slightly shorter than the sequence due to primer3's optimization even when specifying strict positions with the last argument. In this case we choose to ignore it as it will likely not be a problem however if an important motif is left out the promoter will not work very well. To solve this one could screen the input sequence for restriction sites and choose a non-cutter restriction sequence to append to the primer and include a wider piece of the genome to ensure the entire upstream region is extracted.